## Import Essential Library And Get Data Ready

In [ ]:
import os
import re
from tqdm.notebook import tqdm

In [ ]:
from PIL import Image
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPModel, CLIPProcessor
from peft import LoraConfig, get_peft_model, TaskType
from torchvision import transforms

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model_name = "openai/clip-vit-large-patch14"

model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

model = model.to(device)

print("Model and processor loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Model and processor loaded successfully!


In [ ]:
def load_split_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    samples = []
    for line in lines:
        path, label = line.strip().split()
        samples.append((path, int(label)))
    return samples

def load_prompt(file_path):
    id_to_name = {}
    with open(file_path, 'r') as f:
        for line in f:
            idx, name = line.split()[0], re.sub(r'\d+', '', line).strip()
            id_to_name[int(idx)] = name

    return id_to_name

In [ ]:
from google.colab import files
uploaded = files.upload()
!unrar x "caption generation"

Saving caption generation.rar to caption generation.rar

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from caption generation.rar

Creating    caption generation                                        OK
Extracting  caption generation/gemini-long.txt                            33%  OK 
Extracting  caption generation/gemini-short.txt                           49%  OK 
Extracting  caption generation/gpt-long.txt                               76%  OK 
Extracting  caption generation/gpt-short.txt                              92%  OK 
Extracting  caption generation/simple.txt                                 99%  OK 
All OK


In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1ntmFAYessXYRLl8yuD_sPqdyPRtzxwyz
%cd /content/Classification
!tar -xf ip102_v1.1.tar

Retrieving folder contents
Processing file 1EL9TA-J5XsiBR4M3nQkMR_e-2dP48eQN ip102_v1.1.tar
Processing file 1Xa3p1h8jkk7ECHzSuGYrn-d39QOd0ezj classes.txt
Processing file 1kCHN6JKuK-3cSG5jGQQLRX94fAzAeCwPFH-ZsHfs5Jw classes
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1EL9TA-J5XsiBR4M3nQkMR_e-2dP48eQN
From (redirected): https://drive.google.com/uc?id=1EL9TA-J5XsiBR4M3nQkMR_e-2dP48eQN&confirm=t&uuid=8f852486-dd89-4c90-bb66-a1aaf551d74c
To: /content/Classification/ip102_v1.1.tar
100% 3.25G/3.25G [01:09<00:00, 46.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Xa3p1h8jkk7ECHzSuGYrn-d39QOd0ezj
To: /content/Classification/classes.txt
100% 2.82k/2.82k [00:00<00:00, 16.9MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1kCHN6JKuK-3cSG5jGQQLRX94fAzAeCwPFH-ZsHfs5Jw
From (redirected): https://docs.google.com/document/d/1kCHN6JKuK-3cSG5jGQQLRX94f

In [ ]:
dataset_root = "/content/Classification/ip102_v1.1"
images_root = os.path.join(dataset_root, "images")
train_txt = os.path.join(dataset_root, "train.txt")
val_txt = os.path.join(dataset_root, "val.txt")
test_txt = os.path.join(dataset_root, "test.txt")

train_data = load_split_file(train_txt)
test_data = load_split_file(test_txt)
val_data = load_split_file(val_txt)

In [ ]:
def extract_text_clip_features(
    processor,
    model,
    class_texts,
    device,
    save_path
):

    model.eval()
    precomputed_features = []

    for item in tqdm(class_texts, desc="Extracting features"):
        with torch.no_grad():
            # Process text
            text_inputs = processor(
                text=item, return_tensors="pt"
            ).to(device)
            text_outputs = model.text_model(**text_inputs)
            text_pooler_output = text_outputs.pooler_output


        precomputed_features.append({
            'text_features': text_pooler_output.cpu()
        })

        # Free memory with the corrected variable names
        del text_inputs, text_outputs, text_pooler_output
        torch.cuda.empty_cache()

    torch.save(precomputed_features, save_path)
    print(f"Features saved to {save_path}")


In [ ]:
prompts_dict = list(load_prompt('/content/caption generation/gemini-short.txt').values())
extract_text_clip_features(
    processor,
    model,
    prompts_dict,
    device,
    "gemini-short.pt"
)

Extracting features:   0%|          | 0/102 [00:00<?, ?it/s]

Features saved to gemini-short.pt


In [ ]:
prompts_dict = list(load_prompt('/content/caption generation/simple.txt').values())
prompts_dict= [f'a photo of {i}' for i in prompts_dict]
extract_text_clip_features(
    processor,
    model,
    prompts_dict,
    device,
    "simple.pt"
)

Extracting features:   0%|          | 0/102 [00:00<?, ?it/s]

Features saved to simple.pt


In [ ]:
prompts_dict = list(load_prompt('/content/caption generation/gemini-long.txt').values())
extract_text_clip_features(
    processor,
    model,
    prompts_dict,
    device,
    "gemini-long.pt"
)

Extracting features:   0%|          | 0/102 [00:00<?, ?it/s]

Features saved to gemini-long.pt


In [ ]:
def extract_clip_features(
    data,
    processor,
    model,
    images_root,
    device,
    save_path
):

    model.eval()
    precomputed_features = []

    for item in tqdm(data, desc="Extracting features"):
        with torch.no_grad():

            path = os.path.join(images_root, item[0])
            image = Image.open(path).convert("RGB")
            image_inputs = processor(images=image, return_tensors="pt").to(device)
            vision_outputs = model.vision_model(**image_inputs)
            image_feature_vector = vision_outputs.pooler_output

        precomputed_features.append({
            'image_features': image_feature_vector.cpu(),
            'label': item[1]
        })
        del  image_inputs, vision_outputs, image_feature_vector
        torch.cuda.empty_cache()

    torch.save(precomputed_features, save_path)
    print(f"Features saved to {save_path}")

Extracting features: 100%|██████████| 22619/22619 [26:45<00:00, 14.09it/s]


Features saved to test_data.pt


In [ ]:
extract_clip_features(
    train_data,
    processor,
    model,
    images_root,
    device,
    'train_data.pt'
)

Extracting features: 100%|██████████| 45095/45095 [53:05<00:00, 14.16it/s]


Features saved to train_data.pt


In [ ]:
extract_clip_features(
    test_data,
    processor,
    model,
    images_root,
    device,
    'test_data.pt'
)

In [ ]:
extract_clip_features(
    valid_data,
    processor,
    model,
    images_root,
    device,
    'valid_data.pt'
)

In [ ]:
from google.colab import files

file_name = 'gemini-long.pt'
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>